In [25]:
import requests as req
from bs4 import BeautifulSoup
import re
import urllib.parse as urlparse
from urllib.request import urlopen, Request
import urllib.error


In [24]:
url = "https://www.zara.com/in/"
LINKREGEX = "^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$"

In [6]:
page = req.get(url)
soup = BeautifulSoup(page.text,"html.parser")

In [26]:
def url_download_old(url):
    return req.get(url).text

def url_download(url, user_agent="wswp" ,num_retries = 2):
    print("Downloading url : " + str(url))
    headers = {'User-Agent': 'Mozilla/5.0'}
    request = Request(url, headers=headers)
    try:
        html = urlopen(request).read()
        html = html.decode('utf-8')
    except urllib.error.HTTPError as e:
        print('HTTPError: {}'.format(e.code))
        if (e.code >= 500 and e.code < 600 and num_retries>0):
            html = url_download(url,num_retries-1)
        else:
            html = None
    except urllib.error.URLError as e:
        print('URLError: {}'.format(e.reason))
        html = None
    except SocketError as e:
        print('SocketError: {}'.format(e))
        html = None
    except:
        print(e)
    return html

def get_links(html):
    if html is None:
        return []
    else:
        webpage_regex = re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
        return webpage_regex.findall(html)

In [27]:
def link_crawler(seed_url, link_regex):
    crawl_queue = [seed_url]
    seen = set(crawl_queue)
    if(len(seen)==100):
        return seen
    while crawl_queue:
        url = crawl_queue.pop()
        html = url_download(url)
        for link in get_links(html):
            if re.match(link_regex, link):
                link = urlparse.urljoin(seed_url, link)
                if link not in seen:
                    seen.add(link)
                    crawl_queue.append(link)
    return seen

In [28]:
siteurls = link_crawler(url,LINKREGEX)

HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403
HTTPError: 403


NameError: name 'SocketError' is not defined

In [30]:
def crawl_sitemap(url):
    print("Loading site map located at : "+str(url))
    sitemap = url_download(url)
    links = re.findall(r'<loc>(.*?)</loc>',sitemap)
    return links
#     print("Crawling {0} links found in the sitemap".format(len(links)))
#     for link in links:
#         html = url_download(link)

In [31]:
sitemap = "https://www.zara.com/sitemaps/sitemap-index.xml.gz"
crawl_sitemap(sitemap)

Loading site map located at : https://www.zara.com/sitemaps/sitemap-index.xml.gz


NameError: name 'SocketError' is not defined

In [32]:
# https://www.zara.com/sitemaps/sitemap-images-in-en.xml.gz
# ALl images related to India are located here